### Referencias modelo gravitacional

https://es.wikipedia.org/wiki/Modelo_de_gravedad_del_comercio  
https://www.nature.com/articles/s41467-021-26752-4

In [2]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)

### Datos Estudiantes

In [3]:
parvulario=pd.read_csv('parvulario.csv')
escolar=pd.read_csv('escolar.csv')
superior=pd.read_csv('superior.csv')    

In [4]:
educacion=pd.concat([parvulario,escolar,superior])

In [5]:
educacion['comuna']=educacion['comuna'].str.title()
educacion['comuna']=educacion['comuna'].str.strip().str.lower()
educacion=educacion.rename(columns={'ano':'año'})
educacion=educacion.groupby(['comuna', 'año']).agg({'matriculas': 'sum'}).reset_index()

### Datos Laborales

In [6]:
laborales=pd.read_excel('datos_laborales.xls')

In [7]:
laborales_melt=laborales.melt(id_vars='Unidad territorial').query("variable !=' Variable' ").rename(
    columns={'variable':'año','value':'trabajadores','Unidad territorial':'comuna'})

In [8]:
laborales_melt['comuna']=laborales_melt['comuna'].str.strip().str.lower()

In [9]:
laborales_melt['año']=laborales_melt['año'].str.strip().astype(int)

#### Prediccion para el 2024

In [10]:
from sklearn.linear_model import LinearRegression

predicciones = {}
comunas = laborales_melt['comuna'].unique()

for comuna in comunas:
    data_comuna = laborales_melt[laborales_melt['comuna'] == comuna]
    X = data_comuna['año'].values.reshape(-1, 1)
    y = data_comuna['trabajadores'].values
    
    model = LinearRegression()
    model.fit(X, y)
    
    prediccion_2024 = model.predict([[2024]])[0]
    predicciones[comuna] = max(0, prediccion_2024)  

resultados = pd.DataFrame({
    'comuna': list(predicciones.keys()),
    'año': 2024 ,
    'trabajadores': list(predicciones.values())
})

resultados = resultados.sort_values('comuna').reset_index(drop=True)


In [11]:
laborales_melt=pd.concat([laborales_melt,resultados])

### Datos Poblacionales

In [12]:
poblacion=pd.read_excel('estimaciones-y-proyecciones-2002-2035-comunas.xlsx',sheet_name='poblacion_total')

In [13]:
poblacion_melt=poblacion.melt(id_vars='Comuna').query("variable !=2025 ").rename(columns={'variable':'año','value':'poblacion','Comuna':'comuna'})

In [14]:
poblacion_melt['comuna']=poblacion_melt['comuna'].str.strip().str.lower()

### Merge Datasets

In [15]:
df=poblacion_melt.merge(laborales_melt, on=['comuna', 'año'], how='left'). \
merge(educacion, on=['comuna', 'año'], how='left')

#### Considerar desde 2020 en adelante

In [16]:
df=df.query('año >= 2020')

### Leer Comunas

In [17]:
import geopandas as gpd

In [19]:
comunas=gpd.read_file('/Users/mespinoza/Library/Mobile Documents/com~apple~CloudDocs/Magister/Proyecto Final Grado/Dataset/datos/Comunas/comunas.shp')

In [20]:
comunas_rm=comunas.query("Region=='Región Metropolitana de Santiago'")

### Generacion Matriz de Movilidad

In [25]:
df['trabajadores_estudiantes']=df['trabajadores']+df['matriculas']

In [26]:
import geopandas as gpd
import numpy as np
from scipy.spatial import distance_matrix


comunas_rm['geometry'] = comunas_rm.geometry.centroid
comunas_rm['Comuna'] = comunas_rm['Comuna'].str.strip().str.lower()

coords = np.array([(point.x, point.y) for point in comunas_rm['geometry']])

dist_matrix = distance_matrix(coords, coords)

comunas = comunas_rm['Comuna'].tolist()  
dist_df = pd.DataFrame(dist_matrix, index=comunas, columns=comunas)

beta = 2  
K = 1 

years = df['año'].unique()
flow_matrices = {}

for year in years:
    df_year = df[df['año'] == year].set_index('comuna')
    
    flow_matrix = np.zeros((len(comunas), len(comunas)))
    
    for i, comuna_i in enumerate(comunas):
        for j, comuna_j in enumerate(comunas):
            if i == j:
                flow_matrix[i, j] = 0 
            else:
                P_i = df_year.loc[comuna_i, 'poblacion']  
                A_j = df_year.loc[comuna_j, 'trabajadores_estudiantes']  
                D_ij = dist_df.loc[comuna_i, comuna_j] 
                
        
                flow_matrix[i, j] = K * (P_i * A_j) / (D_ij ** beta)
    
    flow_matrices[year] = pd.DataFrame(flow_matrix, index=comunas, columns=comunas)

### Normalizacion de matriz con softmax

In [27]:
def funcion_normalizacion(matriz):
    max_vals = np.max(matriz, axis=1, keepdims=True)
    exp_matriz = np.exp(matriz - max_vals)
    sum_exp = np.sum(matriz, axis=1, keepdims=True)
    softmax_matriz = matriz / sum_exp
    return softmax_matriz

In [28]:
matrices_norm = {}

for year, matrix in flow_matrices.items():
    matrix_values = matrix.values
    normalized_matrix = funcion_normalizacion(matrix_values)
    normalized_df = pd.DataFrame(normalized_matrix, index=matrix.index, columns=matrix.columns)
    matrices_norm[year] = normalized_df


In [29]:
matrices_norm

{2020:                      san joaquín  san miguel  san ramón  independencia  \
 san joaquín             0.000000    0.148711   0.007455       0.005090   
 san miguel              0.080984    0.000000   0.009883       0.005929   
 san ramón               0.023363    0.056871   0.000000       0.004221   
 independencia           0.003061    0.006548   0.000810       0.000000   
 la cisterna             0.024953    0.097596   0.066087       0.005288   
 peñalolén               0.008167    0.011157   0.002767       0.004592   
 providencia             0.006749    0.010666   0.001329       0.018193   
 la reina                0.004223    0.006016   0.001198       0.003873   
 calera de tango         0.006269    0.015364   0.005528       0.005511   
 colina                  0.005048    0.010129   0.002261       0.011505   
 santiago                0.017462    0.040439   0.003032       0.036611   
 lampa                   0.004918    0.010849   0.002348       0.012482   
 pirque            